# Training ResNet-18 with Focal Loss on The Model

## Library Imports

In [3]:
from autogluon.multimodal import MultiModalPredictor
from ray import tune
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import torch
import os

torch.set_float32_matmul_precision("medium")

e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-26 16:40:04,620	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-26 16:40:04,747	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Data Imports

In [4]:
train_df = pd.read_csv("E:/Current_Workdir/palm-fruit-classification/data/clean/train_df.csv")
test_df = pd.read_csv("E:/Current_Workdir/palm-fruit-classification/data/clean/test_df.csv")

## Finetuning ResNet-18

Calculate class proportions for Focal Loss $\alpha$  
see [this](https://amaarora.github.io/posts/2020-06-29-FocalLoss.html) for details.

In [5]:
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(train_df["label_text"]), y=train_df["label_text"])
class_weights_list = class_weights.tolist()
class_weights_list

[2.1965174129353233,
 1.1964769647696476,
 0.44327309236947793,
 5.660256410256411,
 0.5145687645687645,
 3.003401360544218]

Set Training Folder Name

In [29]:
# now = datetime.datetime.now()
# timestamp_str = now.strftime("%Y-%m-%d_%H-%M-%S")
# model_path = "../model/" + f"train_ResNet18_customParams_{timestamp_str}"

In [ ]:
# model_path

In [31]:
# os.makedirs(model_path, exist_ok=True)

Defining Hyperparameters

In [6]:
hyperparameters_model1 = {
    "optimization.learning_rate": 0.000340352,
    "optimization.max_epochs":17,
    "env.batch_size": 8,
    #"env.batch_size": tune.qlograndint(8,128,2),
    "optimization.loss_function": "focal_loss",
    "optimization.focal_loss.alpha": class_weights_list,
    "optimization.focal_loss.gamma": 0.355163,
    "optimization.focal_loss.reduction": "sum",
    "model.timm_image.checkpoint_name": "resnet18",
    "optimization.optim_type": "adamw",
    "optimization.top_k_average_method": "best"
}

hyperparameters_model2 = {
    "optimization.learning_rate": tune.loguniform(0.00005, 0.001),
    "optimization.max_epochs":tune.randint(5,20),
    "env.batch_size": tune.choice([8, 16, 32, 64, 128, 256]),
    #"env.batch_size": tune.qlograndint(8,128,2),
    "optimization.loss_function": "focal_loss",
    "optimization.focal_loss.alpha": class_weights_list,
    "optimization.focal_loss.gamma": tune.uniform(0, 1),
    "optimization.focal_loss.reduction": "sum",
    "model.timm_image.checkpoint_name": "resnet18",
    "optimization.optim_type": "adamw",
    "optimization.top_k_average_method": "best"
}

hyperparameters_model3 = {
    "optimization.learning_rate": tune.loguniform(0.00005, 0.001),
    "optimization.max_epochs":tune.randint(5,20),
    "env.batch_size": tune.choice([8, 16, 32, 64, 128, 256]),
    #"env.batch_size": tune.qlograndint(8,128,2),
    "optimization.loss_function": "focal_loss",
    "optimization.focal_loss.alpha": class_weights_list,
    "optimization.focal_loss.gamma": tune.uniform(0, 1),
    "optimization.focal_loss.reduction": "sum",
    "model.timm_image.checkpoint_name": "resnet18",
    "optimization.optim_type": "adamw",
    "optimization.top_k_average_method": "best"
}

hyperparameters_model4 = {
    "optimization.learning_rate": tune.loguniform(0.00005, 0.001),
    "optimization.max_epochs":tune.randint(5,20),
    "env.batch_size": tune.choice([8, 16, 32, 64, 128, 256]),
    #"env.batch_size": tune.qlograndint(8,128,2),
    "optimization.loss_function": "focal_loss",
    "optimization.focal_loss.alpha": class_weights_list,
    "optimization.focal_loss.gamma": tune.uniform(0, 1),
    "optimization.focal_loss.reduction": "sum",
    "model.timm_image.checkpoint_name": "resnet18",
    "optimization.optim_type": "adamw",
    "optimization.top_k_average_method": "best"
}

hyperparameters_model5 = {
    "optimization.learning_rate": tune.loguniform(0.00005, 0.001),
    "optimization.max_epochs":tune.randint(5,20),
    "env.batch_size": tune.choice([8, 16, 32, 64, 128, 256]),
    #"env.batch_size": tune.qlograndint(8,128,2),
    "optimization.loss_function": "focal_loss",
    "optimization.focal_loss.alpha": class_weights_list,
    "optimization.focal_loss.gamma": tune.uniform(0, 1),
    "optimization.focal_loss.reduction": "sum",
    "model.timm_image.checkpoint_name": "resnet18",
    "optimization.optim_type": "adamw",
    "optimization.top_k_average_method": "best"
}

hyperparameters_model6 = {
    "optimization.learning_rate": tune.loguniform(0.00005, 0.001),
    "optimization.max_epochs":tune.randint(5,20),
    "env.batch_size": tune.choice([8, 16, 32, 64, 128, 256]),
    #"env.batch_size": tune.qlograndint(8,128,2),
    "optimization.loss_function": "focal_loss",
    "optimization.focal_loss.alpha": class_weights_list,
    "optimization.focal_loss.gamma": tune.uniform(0, 1),
    "optimization.focal_loss.reduction": "sum",
    "model.timm_image.checkpoint_name": "resnet18",
    "optimization.optim_type": "adamw",
    "optimization.top_k_average_method": "best"
}

In [14]:
predictor_1 = MultiModalPredictor(label="label_text", problem_type='multiclass', eval_metric="f1_macro")

In [12]:
# hyperparameter_tune_kwargs = {
#     "searcher": "bayes",
#     "scheduler": "ASHA",
#     "num_trials": 1
# }

In [15]:
predictor_1.fit(
        train_data=train_df,
        hyperparameters=hyperparameters_model1,
        # time_limit = 3600
)

No path specified. Models will be saved in: "AutogluonModels\ag-20241226_094305"
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       109.09 GB / 127.91 GB (85.3%)
Disk Space Avail:   1399.39 GB / 1863.00 GB (75.1%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\notebook\AutogluonModels\ag-20241226_094305
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler

Epoch 0:  50%|████▉     | 134/269 [00:04<00:04, 28.66it/s]                 

Epoch 0, global step 134: 'val_f1_macro' reached 0.20254 (best 0.20254), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\notebook\\AutogluonModels\\ag-20241226_094305\\epoch=0-step=134.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:10<00:00, 25.80it/s]

Epoch 0, global step 268: 'val_f1_macro' reached 0.41876 (best 0.41876), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\notebook\\AutogluonModels\\ag-20241226_094305\\epoch=0-step=268.ckpt' as top 3


...

Epoch 16, global step 4572: 'val_f1_macro' reached 0.98189 (best 0.98452), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\notebook\\AutogluonModels\\ag-20241226_094305\\epoch=16-step=4572.ckpt' as top 3


Epoch 16: 100%|██████████| 269/269 [00:11<00:00, 24.44it/s]

`Trainer.fit` stopped: `max_epochs=17` reached.


Epoch 16: 100%|██████████| 269/269 [00:11<00:00, 24.43it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

## Evaluate on Test Dataset

In [17]:
scores = predictor1.evaluate(train_df, metrics=["f1_macro", "f1_micro", "accuracy", "recall_macro", "precision_macro"])
scores

NameError: name 'predictor1' is not defined

In [ ]:
preds = predictor.predict(test_df)

In [13]:
true_labels = test_df['label_text']

In [ ]:
cm = confusion_matrix(true_labels, preds)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=predictor.class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.show()

## Inference Example

In [ ]:
image_path = test_df.iloc[0]['image']
pil_img = Image.open(image_path)  # Use .open() instead of calling Image
display(pil_img)

In [ ]:
predictions = predictor.predict({'image': [image_path]})
predictions

In [ ]:
proba = predictor.predict_proba({'image': [image_path]})
print(proba)

In [ ]:
feature = predictor.extract_embedding({'image': [image_path]})
print(feature[0].shape)

## Save Raw TIMM Format

In [ ]:
predictor.dump_model(model_path + "/timm")